In [1]:
import random
import numpy as np
import tensorflow as tf

max_len = 40

w2idx ={'<eos>': 777777}

/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
    def file_to_ids(file_name):
        with open(file_name, "r") as fin:
            lines = fin.readlines()

        length, ids = [], []
        for num, line in enumerate(lines):
            id = np.zeros(max_len, dtype=np.int32)
            line += " <eos>"
            words = line.split()
            for i, word in enumerate(words):
                if i == max_len:
                    break
                if word not in w2idx:
                    w2idx[word] = len(w2idx)
                id[i] = w2idx[word]
            ids.append(id)
            length.append(i)

            if num == 100000:
                break

        return np.array(ids), np.array(length)

In [3]:
path = './dataset/ptb/ptb.train.txt'

ids, length = file_to_ids(path)

#기존의 dataload와 같은데 sentence의 길이를 40으로 고정하도록 0으로 채우고 sentence의 끝을 지정함
#length는 기존대로 sentence의 길이를 출력

In [8]:
x_input_batch = ids[:,:15]
print(x_input_batch.shape)
print(x_input_batch)
#target_labels = self.decoder_output[:, :self.batch_max_len]


(42068, 15)
[[   1    2    3 ...   13   14   15]
 [  25   26   27 ...   37   38   27]
 [  39   26   40 ...    0    0    0]
 ...
 [4399 5019   35 ... 9999  119  857]
 [  78  835   30 ...    0    0    0]
 [ 108 3622 3623 ...  704 9999  119]]


In [ ]:
import random
import numpy as np

class text_data(object):
    def __init__(self, path="./dataset/ptb", max_len=40, end_token="<eos>"):
        self.train_pt, self.val_pt, self.test_pt = 0, 0, 0
        self.path = path
        self.max_len = max_len

        self.w2idx ={end_token: 0}
        self.train_ids, self.train_len = self.file_to_ids(path+"/ptb.train.txt")
        self.val_ids, self.val_len = self.file_to_ids(path + "/ptb.train.txt")
        self.test_ids, self.test_len = self.file_to_ids(path + "/ptb.train.txt")
        self.vocab_size = len(self.w2idx)

        self.train_size = len(self.train_ids)
        self.val_size = len(self.val_ids)
        self.test_size = len(self.test_ids)

        self.idx2w = {}
        for word in self.w2idx:
            self.idx2w[self.w2idx[word]] = word

    def file_to_ids(self, file_name):
        with open(file_name, "r") as fin:
            lines = fin.readlines()

        length, ids = [], []
        for num, line in enumerate(lines):
            id = np.zeros(self.max_len, dtype=np.int32)
            line += " <eos>"
            words = line.split()
            for i, word in enumerate(words):
                if i == self.max_len:
                    break
                if word not in self.w2idx:
                    self.w2idx[word] = len(self.w2idx)
                id[i] = self.w2idx[word]
            ids.append(id)
            length.append(i)

            if num == 100000:
                break

        return np.array(ids), np.array(length)

    def get_train(self, batch_size=20):
        pt = self.train_pt
        self.train_pt = (self.train_pt + batch_size) % self.train_size
        return self.train_ids[pt: pt+batch_size], self.train_len[pt: pt+batch_size]

    def get_val(self, batch_size=20):
        pt = self.val_pt
        self.val_pt = (self.val_pt + batch_size) % self.val_size
        return self.val_ids[pt: pt+batch_size], self.val_len[pt: pt+batch_size]

    def get_test(self, batch_size=20):
        pt = self.test_pt
        self.test_pt = (self.test_pt + batch_size) % self.test_size
        return self.test_ids[pt: pt+batch_size], self.test_len[pt: pt+batch_size]